# Meta-data for all stations

Reading the 'BE-meta.xml' file, for the stations in Belgium.
All the available XML tags:

In [1]:
import re, collections
from lxml import etree
import xml.etree.ElementTree as ET

tree = ET.parse('BE_meta.xml')
root = tree.getroot()
xml = ET.tostring(root, encoding='utf8', method='xml')
 
xml_root = etree.fromstring(xml)
raw_tree = etree.ElementTree(xml_root)
nice_tree = collections.OrderedDict()
 
for tag in xml_root.iter():
    path = re.sub('\[[0-9]+\]', '', raw_tree.getpath(tag))
    if path not in nice_tree:
        nice_tree[path] = []
    if len(tag.keys()) > 0:
        nice_tree[path].extend(attrib for attrib in tag.keys() if attrib not in nice_tree[path])            
 
for path, attribs in nice_tree.items():
    indent = int(path.count('/') - 1)
    print('{0}{1}: {2} [{3}]'.format('    ' * indent, indent, path.split('/')[-1], ', '.join(attribs) if len(attribs) > 0 else '-'))

0: airbase [{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation]
    1: country [-]
        2: country_name [-]
        2: country_iso_code [-]
        2: country_eu_member [-]
        2: network [-]
            3: network_code [-]
            3: network_name [-]
            3: network_short_name [-]
            3: network_start_date [-]
            3: network_time_reference_basis [-]
            3: network_type [-]
            3: network_manager [-]
                4: organization [-]
                    5: organization_name [-]
                    5: organization_address [-]
                    5: organization_city [-]
                    5: organization_phone_number [-]
                    5: organization_fax_number [-]
                4: person [-]
                    5: person_last_name [-]
                    5: person_first_name [-]
                    5: person_title [-]
            3: network_data_supplier [-]
                4: organization [-]
              

Print all stations with "BE" prefix (Belgian stations). Store lon/lat in dictionary.

In [2]:
stationLonLat = {}
for child in root:    
    for child1 in child:          
        if child1.tag == "station":           
            if child1.attrib['Id'].startswith("BE"):
                for child2 in child1:
                    if child2.tag == "station_info":
                        lon = 0.0
                        lat = 0.0
                        population = 0
                        station_altitude = 0
                        type_of_station = 0
                        station_type_of_area = 0
                        station_characteristic_of_zone = 0
                        station_ozone_classification = 0
                        for child3 in child2:         
                            if child3.tag == "station_latitude_decimal_degrees":
                                lat = float(child3.text)
                            if child3.tag == "station_longitude_decimal_degrees":
                                lon = float(child3.text)
                            if child3.tag == "population":
                                population = float(child3.text)
                            if child3.tag == "station_altitude":
                                station_altitude = float(child3.text)
                            if child3.tag == "type_of_station":
                                type_of_station = ord(child3.text[0]) % 10                             
                            if child3.tag == "station_type_of_area":
                                station_type_of_area = ord(child3.text[0])     % 10                             
                            if child3.tag == "station_characteristic_of_zone":
                                station_characteristic_of_zone = ord(child3.text[0])  % 10                                
                            if child3.tag == "station_ozone_classification":
                                station_ozone_classification = ord(child3.text[0]) % 10 
                                
#                         print ("{} (lon: {}, lat: {}, population: {}, altitude: {}, type_of_station: {} {} {})"
#                                .format(float(lon), float(lat), station_altitude, population, type_of_station, 
#                                 station_type_of_area, station_characteristic_of_zone, station_ozone_classification))
                        stationLonLat[child1.attrib['Id'][:7]] = (
                            float(lon), float(lat), population, station_altitude, type_of_station, 
                            station_type_of_area, station_characteristic_of_zone, station_ozone_classification)

## Station data gathering

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans, KMeans

clusters = 8

coordinates2 = []
for stationName in stationLonLat:
    coordinates2.append((stationLonLat[stationName][0], stationLonLat[stationName][1]))
coordinates2 = np.array(coordinates2)  

coordinates = []
stationNames = []
for stationName in stationLonLat:
    coordinates.append((
        stationLonLat[stationName][0],  # lat
        stationLonLat[stationName][1]  # lon
        ,stationLonLat[stationName][2] # alt
        , stationLonLat[stationName][3] # pop
        , stationLonLat[stationName][4] # station type
        , stationLonLat[stationName][5] # area type
        , stationLonLat[stationName][6] # zone
        , stationLonLat[stationName][7] # ozone class
        
   ))
    stationNames.append(stationName)
coordinates = np.array(coordinates)   

# PM10 station Map

Station names, altitude and population densities:

In [16]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, SplitMapControl, CircleMarker, Popup
from ipywidgets import HTML
#import folium

m = Map(center=(50.5, 5.5), zoom=8)
#m = folium.Map()

#m.layout.width = '1024px'
m.layout.height = '800px'

right_layer = basemap_to_tiles(basemaps.Esri.WorldImagery, "2019-1-11")
left_layer = basemap_to_tiles(basemaps.Esri.WorldTopoMap , "2019-1-11")

control = SplitMapControl(left_layer=left_layer, right_layer=right_layer)
m.add_control(control)

allowedStations = ["BETN043", "BETN045", "BETN052", "BETN054", "BETN060", "BETN063", "BETN066", "BETN067", "BETN070", "BETN073", "BETN085", "BETN093", "BETN100", "BETN113", "BETN121", "BETN132"]

for i in range(len(coordinates)):
    
    if stationNames[i] in allowedStations:
        message = HTML()
        message.value = "<b>" + stationNames[i] + "</b>" + "<br/><i>Altitude:</i> " + str(coordinates[i,3]) + " m" + "<br/><i>Population Density:</i><br/>" + str(coordinates[i,2]) + "/km^2"

        circle_marker = CircleMarker()
        circle_marker.location = (coordinates[i,1], coordinates[i,0])
        circle_marker.radius = int(4 + 10 * (coordinates[i,3]/max(coordinates[:,3])))
        circle_marker.weight = 1
        circle_marker.title = "asdf"
        circle_marker.alt = "asdf"
        circle_marker.color = "blue"
        circle_marker.fill_color = "blue"
        circle_marker.fill_opacity = coordinates[i,3]/max(coordinates[:,3])    
        circle_marker.popup = message
        m.add_layer(circle_marker)    

center = (50.7, 4.8)
message1 = HTML()
message1.value = "<b>Click Circles</b> for station info!<br><b>Opacity & Size </b> = Altitude"
popup = Popup(
    location=center,
    child=message1,
    close_button=True,
    auto_close=False,
    close_on_escape_key=False
)

# Popup associated to a layer
m.add_layer(popup)
m

 # Filepath to the output
#outfp = "pm10_bg_stations_map.html"

# Save the map
#m.save(outfp)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …